# Intent classification
We need to define which class the search query belongs to and distribute the queries between several categories inside the class. There’s a list of queries (related to travel and dining), each with an unknown class and category. Performers are asked to first select the search query’s class and then define the category it belongs to within this class.

To get acquainted with Toloka tools for free, you can use the promo code **TOLOKAKIT1** on $20 on your [profile page](https://toloka.yandex.com/requester/profile?utm_source=github&utm_medium=site&utm_campaign=tolokakit) after registration.

Prepare environment and import all we'll need.

In [107]:
!pip install toloka-kit==0.1.15
!pip install crowd-kit=0.0.7

import datetime
import json
import logging
import sys
import time
import pandas as pd

import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import DawidSkene


logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Сreate toloka-client instance. All api calls will go through it. More about OAuth token in our [Learn the basics example](https://github.com/Toloka/toloka-kit/tree/main/examples/0.getting_started/0.learn_the_basics) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Toloka/toloka-kit/blob/main/examples/0.getting_started/0.learn_the_basics/learn_the_basics.ipynb)

In [ ]:
toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION')  # Or switch to 'SANDBOX'
print(toloka_client.get_requester())

## Creating new project

Use a clear project name and description

In [109]:
project = toloka.Project(
    public_name='Intent classification',
    public_description='Choose the right category and intent for a given text.',
)

Use the Template builder to set the project interface ([more on the Template builder](https://yandex.ru/support/toloka-tb/index.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit))

In [132]:
query_view = tb.AlertViewV1(
    tb.TextViewV1(tb.data.InputData('query')),
    label='QUERY',
    theme='info'
)

domain_choice_field = tb.ButtonRadioGroupFieldV1(
    tb.OutputData('domain'),
    [
        tb.GroupFieldOption('kitchen', 'Kitchen'),
        tb.GroupFieldOption('travel', 'Travel')
    ],
    label='Please select query category:',
    validation=tb.RequiredConditionV1()
)

helper_kitchen = tb.IfHelperV1(
    tb.EqualsConditionV1(
        'kitchen',
        tb.OutputData('domain')
    ),
    tb.RadioGroupFieldV1(
        tb.OutputData('intent'),
        [
            tb.GroupFieldOption(
                'restaurant_reviews',
                'Restaurant review',
                hint='Anything related to whether a restaurant is worth the visit'
            ),
            tb.GroupFieldOption(
                'restaurant_reservation',
                'Restaurant reservation',
                hint='Intention to reserve a table in the restaurant'
            ),
            tb.GroupFieldOption(
                'nutrition_info',
                'Nutritional information',
                hint='Intention to know information about how healthy food or amount of calories'
            ),
            tb.GroupFieldOption(
                'recipe',
                'Recipe',
                hint='Anything related to how to prepare food or food substitution'
            ),
        ],
        label='What is the intent of the query?',
        validation=tb.RequiredConditionV1(hint='please select the relevant intent')
    )
)

helper_travel = tb.IfHelperV1(
    tb.EqualsConditionV1(
        'travel',
        tb.OutputData('domain')
    ),
    tb.RadioGroupFieldV1(
        tb.OutputData('intent'),
        [
            tb.GroupFieldOption(
                'book_hotel',
                'Book a hotel',
                hint='Anything related to reservation a flight or hotel'
            ),
            tb.GroupFieldOption(
                'timezone',
                'Timezone',
                hint='Intention to get information about timezone'
            ),
            tb.GroupFieldOption(
                'travel_suggestion',
                'Travel suggestion',
                hint='Query related to recommendation on a travel'
            ),
            tb.GroupFieldOption(
                'exchange_rate',
                'Exchange rate',
                hint='Intention to know price to buy such currency'
            ),
            tb.GroupFieldOption(
                'vaccines',
                'Vaccine',
                hint='Query which contains information about vaccines in certain region'
            ),
        ],
        label='What is the intent of the query?',
        validation=tb.RequiredConditionV1(hint='please select the relevant intent')
    )
)

task_width_plugin = tb.TolokaPluginV1('scroll', task_width=300)

project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1([query_view, domain_choice_field, helper_kitchen, helper_travel]),
    plugins=[task_width_plugin]
)

Set data specification. And set task interface to project.

In [133]:
input_specification = {'query': toloka.project.StringSpec()}
output_specification = {
    'domain': toloka.project.StringSpec(),
    'intent': toloka.project.StringSpec(),
}

project.task_spec = toloka.project.task_spec.TaskSpec(
    input_spec=input_specification,
    output_spec=output_specification,
    view_spec=project_interface,
)

In [134]:
project.public_instructions = open('./public_instructions/public_instruction.html').read().strip()

Create a project.

In [ ]:
project = toloka_client.create_project(project)

You can go to the project page and check that the interface looks how it's supposed to.

## Training pool
We want to create training to help performers make the tasks better. We will add several training tasks and
require to complete all of them before performing the real tasks.

Training is an essential part of
almost every crowdsourcing
project. It allows you to select
performers who have really
mastered the task, and thus
improve quality. Training is also a
great tool for scaling your task
because you can run it any time
you need new performers ([more on training](https://toloka.ai/docs/guide/concepts/train.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit))

In [ ]:
training_pool = toloka.Training(
    project_id=project.id,
    private_name='Intent Classification training',
    may_contain_adult_content=False,
    assignment_max_duration_seconds=7200,
    mix_tasks_in_creation_order=True,
    shuffle_tasks_in_task_suite=True,
    training_tasks_in_task_suite_count=12,
    retry_training_after_days=10
)

training_pool = toloka_client.create_training(training_pool)

## Main pool

A pool is a set of paid tasks grouped into task pages. These tasks are sent out for completion at the same time.

*Note: All tasks within a pool have the same settings (price, quality control, etc.)*

Specify the [pool parameters.](https://toloka.ai/docs/guide/concepts/pool_poolparams.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit):

Set the price per task suite (for example, $0.01).

*Classification tasks are normally paid as basic tasks because these tasks do not take much time. Read more about [pricing principles](https://toloka.ai/knowledgebase/pricing?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base.*

[Filter](https://toloka.ai/docs/guide/concepts/filters.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) performers who can access the task. Set `may_contain_adult_content` to `False`.

Add filter to choose the Languages and Client options. Set English language as your first filter. This way, performers who speak English will be invited to complete this task. Then specify Toloka web version. These filter will make it possible for performers to complete your task on their computers.



Overlap is the number of users who will complete the same task.  Set an overlap of 3 to get a more confident final label. To understand [how this rule works](https://toloka.ai/docs/guide/concepts/mvote.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit), go to the Requester’s Guide.


Specify the time given to complete a task suite (for example, 600 seconds).

To understand how much time it should take to complete a task suite, try doing it yourself.

In [ ]:
classification_pool = toloka.Pool(
    project_id=project.id,
    # Give the pool any convenient name. You are the only one who will see it.
    private_name='Pool for intent classification',
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),
    reward_per_assignment=0.01,
    auto_accept_solutions=True,
    # time to complete a task suite
    assignment_max_duration_seconds=600,
    # overlap
    defaults=toloka.Pool.Defaults(
        default_overlap_for_new_task_suites=3
    ),
    # Select English-speaking performers with a web/mobile client
    filter=(
        (toloka.filter.Languages.in_('EN')) &
        (toloka.filter.ClientType == 'BROWSER')
    )
)

Use [Smart mixing](https://toloka.ai/docs/guide/concepts/task_upload.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) and specify the number of tasks of each type per page.

We recommend putting as many tasks on one page as a performer can complete in 1 to 5 minutes. That way, performers are less likely to get tired, and they won’t lose a significant amount of data if a technical issue occurs.

To learn more about [grouping tasks](https://toloka.ai/docs/search/?utm_source=github&utm_medium=site&utm_campaign=tolokakit&query=smart+mixing) into suites, read the Requester’s Guide.

In [ ]:
# 4 tasks per page
classification_pool.set_mixer_config(real_tasks_count=3, golden_tasks_count=1)

Attach the training you created earlier and select the accuracy level that is required to reach the main pool.

*Note: This means that Tolokers who get less than 85% accuracy will not see this pool.*

In [ ]:
classification_pool.set_training_requirement(training_pool_id=training_pool.id, training_passing_skill_value=85)

Set up [Quality control](https://toloka.ai/docs/guide/concepts/control.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit). Ban performers who give incorrect responses to control tasks.

*Since tasks such as these have an answer that can be used as ground truth, we can use standard quality control rules like golden sets.*

*Read more about [quality control principles](https://toloka.ai/knowledgebase/quality-control?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in our Knowledge Base or check out [control tasks settings](https://toloka.ai/docs/guide/concepts/goldenset.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit) in the Requester’s Guide.*

Set the number of responses and the percentage of correct responses. Ban performer if their quality in the classification is lower than 80%

In [ ]:
classification_pool.quality_control.add_action(
    collector=toloka.collectors.GoldenSet(),
    conditions=[
        toloka.conditions.GoldenSetCorrectAnswersRate < 80.0,
        toloka.conditions.GoldenSetAnswersCount > 3
    ],
    action=toloka.actions.RestrictionV2(
        scope='PROJECT',
        duration=3,
        duration_unit='DAYS',
        private_comment='Did not answer the control tasks'
    )
)

Set up the up the Fast responses rule. This rule allows you to ban performers who submit tasks at a suspiciously high speed.


In [ ]:
classification_pool.quality_control.add_action(
    collector=toloka.collectors.AssignmentSubmitTime(history_size=5, fast_submit_threshold_seconds=10),
    conditions=[toloka.conditions.FastSubmittedCount > 1],
    action=toloka.actions.RestrictionV2(
        scope='PROJECT',
        duration_unit='PERMANENT',
        private_comment='Fast responses'
    )
)

classification_pool = toloka_client.create_pool(classification_pool)

## Preparing data

This example uses [An Evaluation Dataset for Intent Classification and Out-of-Scope Prediction](https://github.com/clinc/oos-eval)

[![License: CC BY 3.0](https://img.shields.io/badge/License-CC%20BY%203.0-lightgrey.svg)](https://creativecommons.org/licenses/by/3.0/)

Bibtex:
```
@inproceedings{larson-etal-2019-evaluation,
    title = "An Evaluation Dataset for Intent Classification and Out-of-Scope Prediction",
    author = "Larson, Stefan  and
      Mahendran, Anish  and
      Peper, Joseph J.  and
      Clarke, Christopher  and
      Lee, Andrew  and
      Hill, Parker  and
      Kummerfeld, Jonathan K.  and
      Leach, Kevin  and
      Laurenzano, Michael A.  and
      Tang, Lingjia  and
      Mars, Jason",
    booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)",
    year = "2019",
    url = "https://www.aclweb.org/anthology/D19-1131"
}

```

In [ ]:
!curl https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_small.json --output dataset.json

We will work only with these intents: `['restaurant_reviews', 'restaurant_reservation', 'nutrition_info', 'recipe']` and `['book_hotel', 'timezone', 'travel_suggestion', 'exchange_rate', 'vaccines']` (in the domains `kitchen` and `travel` respectively)

In [ ]:
list_of_intents = ['restaurant_reviews', 'restaurant_reservation', 'nutrition_info', 'recipe',
'book_hotel', 'timezone', 'travel_suggestion', 'exchange_rate', 'vaccines']

Let's prepare data for non-golden and golden tasks (control tasks)

Control tasks are tasks that already contain the correct response. They are used for checking the quality of responses from performers. The performer's response is compared to the response you provided. If they match, it means the performer answered correctly.


In [118]:
with open('dataset.json') as f:
    json_file = json.load(f)
    json_data = json_file['train']
    json_golden_data = json_file['val']

# data for non-golden tasks
data = pd.DataFrame(json_data, columns=['query', 'intent'])
# data for golden tasks
golden_data = pd.DataFrame(json_golden_data, columns=['query', 'intent'])

data = data[data['intent'].isin(list_of_intents)].reset_index(drop=True)

golden_data = golden_data[golden_data['intent'].isin(list_of_intents)].reset_index(drop=True)

We will leave only 180 queries for the sake of simplicity

In [119]:
SAMPLE_SIZE = 180
data = data.sample(SAMPLE_SIZE)

data.sample(5)

,query,intent
5,find a suitable lodging in vancouver on march ...,book_hotel
152,show me some things to do in gatlinburg,travel_suggestion
167,tell me what vaccinations i need to get into f...,vaccines
25,exchange rate between mexico and us,exchange_rate
137,do you know what timezone italy is in,timezone


Now we can create non-golden tasks

In [159]:
tasks = [
    toloka.Task(
        pool_id=classification_pool.id,
        input_values={'query': row.query},
    )
    for row in data.itertuples()
]

tasks = toloka_client.create_tasks(tasks, allow_defaults=True)

We'll use 2 queries per intent as a golden task.
In small pools, golden tasks should account for 10–20% of all tasks.

Make sure to include different variations of correct responses in equal amounts.

In [136]:
GOLDEN_SAMPLE_SIZE = 2
golden_data = golden_data.groupby('intent').apply(lambda x: x.sample(GOLDEN_SAMPLE_SIZE)).reset_index(drop=True)

def get_domain(intent):
    if intent in ['restaurant_reviews', 'restaurant_reservation', 'nutrition_info', 'recipe']:
        return 'kitchen'
    else:
        return 'travel'

golden_data['domain'] = golden_data.apply(lambda row: get_domain(row['intent']), axis=1)
golden_data.sample(5)

,query,intent,domain
9,book a reservation for 3 at xenophobe under th...,restaurant_reservation,kitchen
6,i would like to learn to make cookies,recipe,kitchen
14,can you suggest some of the most popular trave...,travel_suggestion,travel
17,do i have to get shots to enter cuba,vaccines,travel
3,convert 100 dollars to euros,exchange_rate,travel


Let's create golden tasks

In [160]:
golden_tasks = [
    toloka.Task(
        pool_id=classification_pool.id,
        input_values={'query': row.query},
        known_solutions = [
            toloka.task.BaseTask.KnownSolution(
                output_values={'domain': row.domain, 'intent': row.intent}
            )
        ],
        infinite_overlap=True,
    )
    for row in golden_data.itertuples()
]

golden_tasks = toloka_client.create_tasks(golden_tasks, allow_defaults=True)

Let's create training tasks with hints for users (based on the same [dataset](https://github.com/clinc/oos-eval))

It’s important to include examples for all classes in the training. Make sure the training set is balanced and the comments explain why an answer is correct. Don’t just name the correct answers.

In [161]:
training_data = [
 ['are there a reservation available at xenophone',
  'kitchen',
  'restaurant_reservation',
  'It is reservation restaurant class in kitchen category. Try to find key words:  reserve / book , name of the restaurant'],
 ['I need a table for 7 pm under the name Paul',
  'kitchen',
  'restaurant_reservation',
  'Query can consist of key words like  table, time, name. Category kitchen, class reservation'],
 ['what are the review for burger king',
  'kitchen',
  'restaurant_reviews',
  'Pay attention to existence of the name of restaurant or the review  key word. Category kitchen, class review'],
 ['can I sub ketchup for mayo',
  'kitchen',
  'recipe',
  'Anything related to substitution. Key words:  instead, substitution, sub. Category kitchen, class recipe'],
 ['pull up soup recipe',
  'kitchen',
  'recipe',
  "One of common words for such class could be 'recipe'. Category kitchen, class recipe"],
 ['How healthy is orange',
  'kitchen',
  'nutrition_info',
  "'healthy' word is a marker of nutrition class. Category kitchen, class nutrition"],
 ['How many calories are in mushrooms',
  'kitchen',
  'nutrition_info',
  'Select this class if something is said about the amount of calories. Category kitchen, class nutrition'],
 ['in which time zone does Denver reside',
  'travel',
  'timezone',
  'You can notice basic words like zone and time. As a result, category is travel and class is timezone'],
 ['find me a hotel with good reviews in phoenix',
  'travel',
  'book_hotel',
  'If you find the word hotel than it could be a sign of category travel and class book a hotel'],
 ["what's the best place for travelling to this time of year",
  'travel',
  'travel_suggestion',
  'Intent which consists of any recommendations about traveling. Category travel, class travel suggestion'],
 ['find me the exchange rate between usd and cad',
  'travel',
  'exchange_rate',
  'Type of currency written in the query suggests category travel with class exchange rate'],
 ['are more shots needed to travel to argentina',
  'travel',
  'vaccines',
  'Intent with shots and vaccines - key words related to category travel and class vaccine']
]
training_dataset = pd.DataFrame(training_data,
                              columns=['query', 'domain', 'intent', 'hint']
)

In [162]:
training_tasks = [
    toloka.Task(
        input_values={'query': row.query},
        known_solutions=[toloka.task.BaseTask.KnownSolution(
            output_values={'domain': row.domain, 'intent': row.intent}
        )],
        message_on_unknown_solution=row.hint,
        infinite_overlap=True,
        pool_id=training_pool.id
    )
    for row in training_dataset.itertuples()
]

training_tasks = toloka_client.create_tasks(training_tasks, allow_defaults=True)

## Open pools

We recommend opening the training pool along with the main pool. Otherwise Tolokers will spend their time on training but get no access to real tasks, which is frustrating. Also, do not forget to close the training pools when there are no main tasks available anymore.

In [ ]:
toloka_client.open_pool(training_pool.id)
toloka_client.open_pool(classification_pool.id)

## Receiving responses

Wait until the pool is completed.

In [ ]:
def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Pool {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(classification_pool.id)
toloka_client.close_training(training_pool.id)

Get responses

When all the tasks are completed, look at the responses from performers.

In [164]:
answers = []

answers_df = toloka_client.get_assignments_df(classification_pool.id)
# Prepare DataFrame
answers_df = answers_df.rename(columns={
    'INPUT:query': 'task',
    'ASSIGNMENT:worker_id': 'performer'
})
# Create a feature-string for aggregation that contains both task's domain and intent
answers_df['label'] = answers_df['OUTPUT:domain'] + ' ' + answers_df['OUTPUT:intent']

print(f'answers count: {len(answers_df)}')

[WARNING] toloka.client: Experimental method
answers count: 216


Aggregation results using the Dawid-Skene

We use this aggregation model because our questions are of comparable difficulty, and we don't have many control tasks.

Read more about [the Dawid-Skene model](https://toloka.ai/docs/guide/concepts/result-aggregation.html?utm_source=github&utm_medium=site&utm_campaign=tolokakit#aggr__dawid-skene) in the Requester’s Guide or get at an overview of different [aggregation models](https://toloka.ai/knowledgebase/aggregation?utm_source=github&utm_medium=site&utm_campaign=tolokakit) our Knowledge Base.

In [176]:
# Run aggregation
predicted_answers = DawidSkene(n_iter=20).fit_predict(answers_df)

Once the aggregation completes, let's look at some results

In [179]:
pd.DataFrame(predicted_answers.sample(15), columns=['domain and intent'])

,domain and intent
task,
book a reservation for 3 at xenophobe under the name zebee,kitchen restaurant_reservation
"now, i need a reservation for backwoods crossing at seven for 6 people",kitchen restaurant_reservation
how many pesos can i get for one dollar,travel exchange_rate
how can i make chicken pot pie,kitchen recipe
i need you to find me a recipe for fried shrimp,kitchen recipe
i wanna know five dollars in yen and rubles,travel exchange_rate
how's the sausage pizza at pizza hut,kitchen restaurant_reviews
do you know the nutrition facts for grapes,kitchen nutrition_info
can you suggest some of the most popular travel destination,travel travel_suggestion
